# Clean Inventarios

In [1]:
import re
import warnings
import pandas as pd
import numpy as np
import openpyxl as xl
from excel_functions import ExcelFunctions
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side

In [2]:
def cargar_libro_excel_sin_warning(nombre_archivo, data_only=True):
    # Ignorar temporalmente el warning durante la carga del libro de Excel
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=UserWarning)
        libro_excel = xl.load_workbook(nombre_archivo, data_only=data_only)

    return libro_excel

In [3]:
def extraer_fecha_from_nombre(nombre):
    # Definimos una lista de posibles patrones de fecha que podrían aparecer en el nombre
    patrones_fecha = [
        r'\b\d{1,2}-\d{1,2}-\d{4}\b',   # Formato dd-mm-yyyy
        r'\b\d{1,2}/\d{1,2}/\d{4}\b',   # Formato dd/mm/yyyy
        r'\b\d{4}-\d{1,2}-\d{1,2}\b',   # Formato yyyy-mm-dd
        r'\b\d{4}/\d{1,2}/\d{1,2}\b',   # Formato yyyy/mm/dd
    ]

    # Buscamos los patrones en el nombre y extraemos la primera coincidencia de fecha
    fecha = None
    for patron in patrones_fecha:
        coincidencias = re.findall(patron, nombre)
        if coincidencias:
            fecha = coincidencias[0]
            break

    return fecha

In [4]:
def clean_inventory_file(file, dir_location, saving_name=None, return_data=True):
    '''
    Esta función recibe un archivo, ya sea ubicación para abrirlo o bytes y lo limpia.
    dir_location: el directorio donde lo va a guardar
    '''
    # Definimos parámetros necesarios
    sheet_name = 'Rep Prod Terminado Ubi Chh Mx'
    wanted_cols = ['CLAVE', 'Descripción', 'Total x Producto']
    column_output_names = ['sku', 'descripcion', 'inventario']

    # Creamos el objeto de funciones de excel
    ef = ExcelFunctions()

    # Abrimos el archivo pedido
    wb = cargar_libro_excel_sin_warning(file, data_only=True)
    ws = wb[sheet_name]
    fecha = extraer_fecha_from_nombre(file)

    # Encontramos la fila con el nombre de las columnas
    row_with_col_names = ef.find_value(ws, 1, text='CLAVE')[0]
    last_row_with_data = ef.encontrar_ultima_fila_con_datos(ws, 1, 5)

    # Encontramos los nombres de las columnas en esa fila
    dict_cols = ef.get_columns_on_row(ws, row_with_col_names, num_nones_consecutivos=3)

    # Sacamos los datos de las columnas deseadas
    extracted_data = ef.get_data_on_columns(
        ws, 
        row_with_col_names+1, 
        dict_cols, 
        ending_row=last_row_with_data, 
        cols_to_extract=wanted_cols,
        output_columns=column_output_names
    )

    # Procesamos un poco los datos
    extracted_data = (extracted_data
     .assign(
         fecha=fecha,
         inventario=lambda x: pd.to_numeric(x.inventario, errors='coerce')
     )
     [['fecha', 'sku', 'descripcion', 'inventario']]
    )
    
    # Guardamos el archivo
    if saving_name is None:
        saving_name = f'inventario_{fecha.replace("-", "_")}'
        
    bytes_data, filename = ef.save_and_download_excel_file(
        extracted_data,
        dir_location=dir_location,
        file_name=saving_name,
        sheet_name='Inventario',
        n_cols_to_bold=3,
        return_data=return_data
    )
    
    if return_data:
        return bytes_data, filename
    
    print(f'{saving_name} was updated succesfully')

In [5]:
clean_inventory_file(
    '../raw_data/DISTRIBUCIÓN Y ABASTO 31-07-2023.xlsx',
    '../transformed_data',
    return_data=False
)

inventario_31_07_2023 was updated succesfully
